In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ml_metrics import quadratic_weighted_kappa
import xgboost as xgb
import datetime as dt
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
import functools
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from scipy import optimize

from xgboostmodel import XGBoostModel, ModelPrediction

# Step 03.  Optimized classification

### Goal 1: Implement class which allows easy combination of boosters to make predictions

### Goal 2: Optimize the classify function, that converst scores to categories

We start out by fitting our best `reg:linear` and `multi:softmax` models which we obtained in Step 02:

In [2]:
booster = XGBoostModel(nfolds=3)

fold = 1 # the fold to use for train-test split

booster.learn_model(fold, objective='reg:linear', num_round=250, 
                    make_plot=False,
                    eta=0.06,
                    max_depth=9,
                    min_child_weight=150,
                    colsample_bytree=0.8,
                    subsample=0.8)

booster.learn_model(fold, objective='multi:softmax', num_round=250, 
                    make_plot=False,
                    eta=0.06,
                    max_depth=8,
                    min_child_weight=50,
                    colsample_bytree=0.8,
                    subsample=0.8)

booster.get_scores()[['objective', 'train_qwk', 'test_qwk']]

Optimization terminated successfully.
         Current function value: 0.273198
         Iterations: 11
         Function evaluations: 1168


,objective,train_qwk,test_qwk
0,reg:linear,0.726802,0.649477
1,multi:softmax,0.615971,0.550655


# We define the ComboPredict class

In [3]:
#===============================================================================

class ComboPredict:
    
    def __init__(self, booster):
        """
        Intitialize the combination predictor with an XGBoostModel instance
        
        Parameters
        ----------
        
        booster : XGBoostModel
        
        """
        
        self.booster = booster
        
        if (len(self.booster.models) == 0):
            raise ValueError("The XGBoostModel provided does not contain any "
                             "fitted models.")
        
        
    def predict_score(self, features, overall_cls_factor):
        """
        Predicts scores for a set of observations.  The score is calculated
        by weighting all of the models present in the XGBoostModel that is 
        passed into the class at initialization. 
        
        Note that the scores output by this function need to be coerced to
        category values for a final prediction. 
        
        Parameters
        ----------
        
        features : array
        
            Features for which predictions weill be generated
            
        overall_cls_factor : float
        
            Relative weight of the classification boosters wrt to the regression
            boosters
            
        """
        
        xg_input = xgb.DMatrix(features)

        weighted_preds = []
        norms = []
        for m in zip(booster.models, booster.scores):
            model, model_fold, model_pred = m[0]
            score = m[1]

            nfeatures = xg_input.num_row()
            X, _ = np.meshgrid(np.arange(8), np.arange(nfeatures))

            if score['objective'] == 'multi:softmax':
                pred_cls = model.predict(xg_input,
                                         ntree_limit=model.best_iteration)
                dummies = pd.get_dummies(pred_cls).values
                weight = model_pred.precisiontrain.reshape(8,1) 

                weighted_pred = np.dot(X * dummies, overall_cls_factor * weight)
                weighted_preds.append(weighted_pred)

                norm = np.dot(dummies, overall_cls_factor * weight )
                norms.append(norm)

            else:
                reg_pred = model.predict(xg_input,
                                         ntree_limit=model.best_iteration)
                weighted_preds.append(reg_pred.reshape(nfeatures, 1))
                norms.append(np.ones_like(weighted_preds))

        total = np.sum(weighted_preds, axis=0) 
        norm = np.sum(norms, axis=0)

        combo_score = np.squeeze(total / norm)
        return combo_score

We demonstrate the use of `ComboPredict` to calculate scores:

In [4]:
combo = ComboPredict(booster)

# All of the models in booster were trained with fold=1:
assert all(fold == 1 for _, fold, _ in booster.models)

# We get the features and labels for fold=1
train, test = booster.make_cv_split(1, returnxgb=False)
# train, test are each a tuple of the form (features, labels)

overall_cls_factor = 0.4
score_train = combo.predict_score(train[0], overall_cls_factor)
score_test = combo.predict_score(test[0], overall_cls_factor)

We then use rounding to the neareset integer to produce categories, and we check kappa for the results:

In [5]:
def classify(score):
    score = np.asarray(score)
    return np.rint(np.clip(score, -0.49, 7.49))

yhcombotrain = classify(score_train)
yhcombotest = classify(score_test)

print("train qwk = {:0.5f}".format(quadratic_weighted_kappa(yhcombotrain, 
                                                            train[1])))

print(" test qwk = {:0.5f}".format(quadratic_weighted_kappa(yhcombotest, 
                                                            test[1])))

train qwk = 0.67922
 test qwk = 0.60933


We recover the results of Step 02, where it was shown that using a weighing ratio 0.4:1 (classification:regression) contribution to the score was beneficial for the quadratic weighted kappa of our predictions. 

# Classification with cutoffs

Rather than rounding to the nearest integer, we implement a function that allows setting score cuttofs which map to each of the eight possible categories: 

In [6]:
def classify_with_cutoffs(yscore, cutoffs):
    """
    Receives a list of seven cutoffs, which will determine the mapping from 
    scores to categories.
    
    Parameters
    ----------
    
    predicted_score : array
    
        Array of predicted scores, which will be mapped onto categories 
        according to cutoffs
    
    cutoffs : array 
    
        Array of length 7 (num_categories - 1).  
    """
    assert len(cutoffs) == 7
    cutoffs = np.sort(cutoffs)
    return np.digitize(yscore, cutoffs).astype('int')
    
    
print("EXAMPLES OF classify_with_cutoffs:\n")
cutoffs0 = np.arange(7)+0.5
for val in list(np.random.rand(3)*9. -2.) + [5.49, 5.51]:
    cat = classify_with_cutoffs(val, cutoffs0)
    print("score={:.2f} => category={}".format(val, cat))

EXAMPLES OF classify_with_cutoffs:

score=0.97 => category=1
score=4.48 => category=4
score=3.84 => category=4
score=5.49 => category=5
score=5.51 => category=6


### We define a function for optimizing the cutoffs:

The cutoffs get "trained" based on the calculated score and the known true labels.

In [13]:
def optmize_cutoffs_simplex(yscore, ytrue, errorfun, *, verbose=False):
    """
    Receives an array of predicted scores, and an array of true values. 
    Determines which cutoff values make for the best prediction with
    respect to the true values. 
    
    Parameters
    ----------
    
    yscore : array
        
        Array of predicted scores
        
    ytrue : array
    
        Array of true  values 
        
    verbose : bool (optional) 
    
        When true prints the std dev of y-ypred before and after 
        optimization.   
    """
    
    yscore = np.asarray(yscore, dtype=np.float64)
    ytrue = np.asarray(ytrue, dtype=np.float64)
    
    def error(p):
        return errorfun(p, yscore, ytrue)
    
    cutoffs0 = np.arange(7)+0.5
    
    just = 15
    if verbose:
        print("{} : {}".format(
                "start error".rjust(just), error(cutoffs0)))
    
    #xopt, fopt, niter, funcalls, warnflag, allvecs 
    pfit = optimize.fmin_powell(error, cutoffs0, xtol=1e-2, ftol=1e-6, 
                                maxiter=None, maxfun=None)
    
    if verbose:
        print("{} : {}\n".format(
                "final error".rjust(just), error(pfit)))
        
    return np.sort(pfit)


def error_lstsq(p, yscore, ytrue):
    errors = classify_with_cutoffs(yscore, p).astype(np.float64) - ytrue
    return np.std(errors)

print("UNOPTIMIZED:")
print("\n qwk = {:0.4f}\n".format(
        quadratic_weighted_kappa(classify(score_train).astype(np.int64), train[1])
    ))

print("="*80)
pfit = optmize_cutoffs_simplex(score_train, train[1], error_lstsq, verbose=True)
print("OPTIMIZED CUTOFFS LSTSQ:")
print(pfit)
bestpfit_leastsq = pfit
print("\n qwk = {:0.4f}\n".format(
        quadratic_weighted_kappa(classify_with_cutoffs(score_train, bestpfit_leastsq), train[1])
    ))


def error_qwk(p, yscore, ytrue):
    errors = quadratic_weighted_kappa(classify_with_cutoffs(yscore, p).astype(np.int64), ytrue)
    return 1 - errors

print("="*80)
pfit = optmize_cutoffs_simplex(score_train, train[1], error_qwk, verbose=True)
print("OPTIMIZED CUTOFFS QWK:")
print(pfit)
bestpfit_qwk = pfit
print("\n qwk = {:0.4f}\n".format(
        quadratic_weighted_kappa(classify_with_cutoffs(score_train, bestpfit_qwk), train[1])
    ))


UNOPTIMIZED:

 qwk = 0.6792

    start error : 1.704812979226756
Optimization terminated successfully.
         Current function value: 1.696007
         Iterations: 9
         Function evaluations: 839
    final error : 1.6960072842910539

OPTIMIZED CUTOFFS LSTSQ:
[ 0.32229194  1.63951265  2.67249365  3.63436993  4.76973765  5.63572747
  6.37087037]

 qwk = 0.6934

    start error : 0.3207754330380226
Optimization terminated successfully.
         Current function value: 0.276877
         Iterations: 9
         Function evaluations: 808
    final error : 0.2768769249311198

OPTIMIZED CUTOFFS QWK:
[ 1.8277727   2.67539148  3.26617627  3.9487634   4.79714536  5.46358664
  6.07641931]

 qwk = 0.7231



In [9]:
print(quadratic_weighted_kappa(classify_with_cutoffs(score_train, pfit).astype(np.int64), train[1]))

0.7231230750688802


In [14]:
yhcombotrain = classify(score_train)
yhcombotest = classify(score_test)

print("="*80)
print("UNOPTIMIZED:")
print("train qwk = {:0.5f}".format(quadratic_weighted_kappa(yhcombotrain, 
                                                            train[1])))
print(" test qwk = {:0.5f}".format(quadratic_weighted_kappa(yhcombotest, 
                                                            test[1])))

yhcombotrain = classify_with_cutoffs(score_train, bestpfit_leastsq)
yhcombotest = classify_with_cutoffs(score_test, bestpfit_leastsq)
print("="*80)
print("OPTIMIZED LEASTSQ:")
print("train qwk = {:0.5f}".format(quadratic_weighted_kappa(yhcombotrain, 
                                                            train[1])))
print(" test qwk = {:0.5f}".format(quadratic_weighted_kappa(yhcombotest, 
                                                            test[1])))

yhcombotrain = classify_with_cutoffs(score_train, bestpfit_qwk)
yhcombotest = classify_with_cutoffs(score_test, bestpfit_qwk)
print("="*80)
print("OPTIMIZED QWK:")
print("train qwk = {:0.5f}".format(quadratic_weighted_kappa(yhcombotrain, 
                                                            train[1])))
print(" test qwk = {:0.5f}".format(quadratic_weighted_kappa(yhcombotest, 
                                                            test[1])))

UNOPTIMIZED:
train qwk = 0.67922
 test qwk = 0.60933
OPTIMIZED LEASTSQ:
train qwk = 0.69342
 test qwk = 0.61843
OPTIMIZED QWK:
train qwk = 0.72312
 test qwk = 0.64827


# Predict for submission

We know go back to the submission samples and make a prediction based on our current model.

In [15]:
data = pd.read_csv('csvs/data_imputed.csv')
features = data[data['train?'] == True].drop(['train?', 'Id', 'Response'], 
                                             axis=1)

labels = data[data['train?'] == True]['Response'].astype('int') -1 

submission_features = data[data['train?'] == False].drop(['train?', 
                                                          'Id', 
                                                          'Response'], 
                                                         axis=1)

combo = ComboPredict(booster)

# All of the models in booster were trained with fold=1:
assert all(fold == 1 for _, fold, _ in booster.models)

overall_cls_factor = 0.4
score_submission = combo.predict_score(submission_features, overall_cls_factor)

best_cutoffs = np.array([ 0.3223,  1.6395,  2.6725,  3.6344,  
                         4.7697,  5.6357,  6.3709])
yhcombo_submission = classify_with_cutoffs(score_submission, best_cutoffs)

submission_ids = data[data['train?'] == False]['Id']
submission_df = pd.DataFrame({"Id": submission_ids, 
                              "Response": yhcombo_submission.astype('int') + 1})

submission_df = submission_df.set_index('Id')
submission_df.to_csv('step03_submission_lstsq.csv')

submission_df.describe()

,Response
count,19765.000000
mean,5.721882
std,1.804223
min,1.000000
25%,5.000000
50%,6.000000
75%,7.000000
max,8.000000


The submission obtained above scores 0.63806 in Kaggle's leaderboard.   The best score at the moment (02/06 at 19:08) is 0.68271.

In [16]:
data = pd.read_csv('csvs/data_imputed.csv')
features = data[data['train?'] == True].drop(['train?', 'Id', 'Response'], 
                                             axis=1)

labels = data[data['train?'] == True]['Response'].astype('int') -1 

submission_features = data[data['train?'] == False].drop(['train?', 
                                                          'Id', 
                                                          'Response'], 
                                                         axis=1)

combo = ComboPredict(booster)

# All of the models in booster were trained with fold=1:
assert all(fold == 1 for _, fold, _ in booster.models)

overall_cls_factor = 0.4
score_submission = combo.predict_score(submission_features, overall_cls_factor)

best_cutoffs = np.array([1.8277727, 2.67539148, 3.26617627, 3.9487634, 
                         4.79714536, 5.46358664, 6.07641931])
yhcombo_submission = classify_with_cutoffs(score_submission, best_cutoffs)

submission_ids = data[data['train?'] == False]['Id']
submission_df = pd.DataFrame({"Id": submission_ids, 
                              "Response": yhcombo_submission.astype('int') + 1})

submission_df = submission_df.set_index('Id')
submission_df.to_csv('step03_submission_qwk.csv')

submission_df.describe()

,Response
count,19765.000000
mean,5.606628
std,2.251880
min,1.000000
25%,4.000000
50%,6.000000
75%,8.000000
max,8.000000


The submission obtained here (`step03_submission_qwk.csv`) scores 0.66193 on Kaggle's leaderboard.  The best score at the moment 02/13 at 7:40 AM is 0.68325.  

Spots 172 to 435 in the leaderboard are all tied at 0.67459.